#### Example: WebSearch as a Tool
1) Prompt -> LLM.
2) LLM decides a web search is needed.
3) Call WebSearch tool
4) Augment user's prompt with search results
5) Generate answer

In [7]:
from dotenv import load_dotenv
import os
from tavily import TavilyClient
from typing import List
import dspy

load_dotenv()

True

In [8]:
tavily_client = TavilyClient(api_key=os.getenv('TAVILY_API_KEY'))

def fetch_recent_news(query: str) -> List[str]:
    """
    Inputs a query string, searches for news, and returns the top results.
    """
    
    response = tavily_client.search(query=query, topic='news', max_results=4)
    return [
        x['content']
        for x in response['results']
    ]

class HaikuGenerator(dspy.Signature):
    """Generates a haiku about the latest news on the query."""
    query = dspy.InputField()
    summary = dspy.OutputField(desc='A summary of the latest news.')
    haiku = dspy.OutputField()

In [ ]:
program = dspy.ReAct(
    signature=HaikuGenerator,
    tools=[fetch_recent_news],
    max_iters=1,  # number of tool calls that the LLM can make
)
program.set_lm(dspy.LM('gemini/gemini-2.5-flash-lite', temperature=0.7))
pred = program(query='OpenAI')

In [13]:
print(pred.summary)
print()
print(pred.haiku)

Recent news about OpenAI includes Elon Musk's lawsuit alleging a violation of its founding mission, which will proceed to trial. The company is also reportedly developing a new voice AI model and devices, and has launched ChatGPT Health, a feature for analyzing medical test results and offering health advice.

Musk's suit goes to trial,
Voice AI, health insights bloom,
Future takes new form.


In [14]:
print(program.inspect_history(n=4))





[2026-01-08T13:57:25.587793]

System message:

Your input fields are:
1. `query` (str): 
2. `trajectory` (str):
Your output fields are:
1. `next_thought` (str): 
2. `next_tool_name` (Literal['fetch_recent_news', 'finish']): 
3. `next_tool_args` (dict[str, Any]):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## query ## ]]
{query}

[[ ## trajectory ## ]]
{trajectory}

[[ ## next_thought ## ]]
{next_thought}

[[ ## next_tool_name ## ]]
{next_tool_name}        # note: the value you produce must exactly match (no extra characters) one of: fetch_recent_news; finish

[[ ## next_tool_args ## ]]
{next_tool_args}        # note: the value you produce must adhere to the JSON schema: {"type": "object", "additionalProperties": true}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Generates a haiku about the latest news on the query.
        
        You are an Agent. In each episode, you will be given the f

Key fragment:
```ini
[[ ## next_tool_name ## ]]
fetch_recent_news

[[ ## next_tool_args ## ]]
{"query": "OpenAI"}

```

MCP Servers
- Not covered here.
- Standardized protocol of providing context and tools to LLMs.

Popularity of Tools
- Tools in Gemini CLI: `ReadFolder`, `ReadFile`, `SearchText`, `FindFiles`, `Edit`, `WriteFile`, `WebFetch`, `ReadManyFiles`, `Shell`, `SaveMemory`, `GoogleSearch`

### WriteFile example

In [15]:
class HaikuGenerator(dspy.Signature):
    """
    Generates a haiku about the latest news on the query.
    Saves the final summary in a file.
    """
    query = dspy.InputField()
    summary = dspy.OutputField(desc='A summary of the latest news.')
    haiku = dspy.OutputField()

def write_file(text: str, filename: str):
    """Write text into a file."""
    with open(filename,'w') as f:
        f.write(text)

In [ ]:
program = dspy.ReAct(
    signature=HaikuGenerator,
    tools=[fetch_recent_news, write_file],
    max_iters=2,  # need to increase this so it can call 2 tools
)
program.set_lm(dspy.LM('gemini/gemini-2.5-flash-lite', temperature=0.7))
pred = program(query='OpenAI')

In [20]:
print(pred.summary)

Recent news about OpenAI includes a lawsuit filed by Elon Musk, alleging the company violated its founding mission by becoming a for-profit entity. A judge has allowed this case to proceed to trial. Additionally, OpenAI is reportedly developing a new voice AI model for voice-based devices, with plans to potentially ship millions of units. The company has also launched ChatGPT Health, a new feature allowing users to analyze medical test results, get dietary advice, and prepare for doctor's appointments, marking a significant expansion into the healthcare sector.


In [21]:
program.inspect_history(n=4)





[2026-01-08T14:11:05.101507]

System message:

Your input fields are:
1. `query` (str): 
2. `trajectory` (str):
Your output fields are:
1. `next_thought` (str): 
2. `next_tool_name` (Literal['fetch_recent_news', 'write_file', 'finish']): 
3. `next_tool_args` (dict[str, Any]):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## query ## ]]
{query}

[[ ## trajectory ## ]]
{trajectory}

[[ ## next_thought ## ]]
{next_thought}

[[ ## next_tool_name ## ]]
{next_tool_name}        # note: the value you produce must exactly match (no extra characters) one of: fetch_recent_news; write_file; finish

[[ ## next_tool_args ## ]]
{next_tool_args}        # note: the value you produce must adhere to the JSON schema: {"type": "object", "additionalProperties": true}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Generates a haiku about the latest news on the query.
        Saves the final summary in a file.
      

### JokeGenerator example

In [22]:
import asyncio
from typing import Optional
from pydantic import BaseModel, Field

In [23]:
class JokeIdea(BaseModel):
    setup: str
    contradiction: str
    punchline: str

class QueryToIdea(dspy.Signature):
    """You are a funny comedian and your goal is to generate a nice structure for a joke."""
    query: str = dspy.InputField()
    joke_idea: JokeIdea = dspy.OutputField()

class IdeaToJoke(dspy.Signature):
    """
    You are a funny comedian who likes to tell stories before delivering a punchline.
    You are always funny and act on the input joke idea.
    If you are provided a draft of a joke, your goal should be to make it funnier and punchy.
    """
    joke_idea: JokeIdea = dspy.InputField()
    joke_draft: Optional[str] = dspy.InputField(description='An existing joke that you need to either refine or change.')
    joke: str = dspy.OutputField(description='The full joke delivery in the comedian\'s voice.')

class JokeJudge(dspy.Signature):
    """Rank each joke idea between 1 to N. Rank 1 is the most unique and funniest."""
    joke_idea: List[JokeIdea] = dspy.InputField()
    joke_ratings: List[int] = dspy.OutputField(description='Rank between 1, 2, 3, ..., N')

def check_score_goodness(args, pred):
    n_samples = len(args['joke_idea'])
    same_len = len(pred.joke_ratings) == n_samples
    all_ranks_present = all([
        (i+1) in pred.joke_ratings
        for i in range(n_samples)
    ])
    return 1 if (same_len and all_ranks_present) else 0

In [24]:
class IdeaGenerator(dspy.Module):
    def __init__(self, n_samples=3):
        # only need to replace ChainOfThought with ReAct and pass the tools
        self.query2idea = dspy.ReAct(
            QueryToIdea,
            [fetch_recent_news],
            1
        )
        self.query2idea.set_lm(dspy.LM('gemini/gemini-2.5-flash-lite', temperature=1))
        
        self.judge = dspy.Refine(dspy.ChainOfThought(JokeJudge),
                                 N=3, reward_fn=check_score_goodness,
                                 threshold=1)
        self.judge.set_lm(dspy.LM('gemini/gemini-2.5-flash-lite', temperature=1))
        
        self.n_samples = n_samples
    
    async def acall(self, query: str) -> JokeIdea:
        joke_ideas = await asyncio.gather(*[
            self.query2idea.acall(query=query)
            for _ in range(self.n_samples)
        ])
        
        print(f'Generated Joke Ideas: \n{joke_ideas}')
        
        judge_score = self.judge(joke_idea=joke_ideas).joke_ratings
        print(f'Judge score for each: {judge_score}')
        
        best_idx = judge_score.index(1)
        selected = joke_ideas[best_idx]
        print(f'Selected: \n{selected}')
        
        return selected

class JokeGenerator(dspy.Module):
    def __init__(self, n_reflections=3):
        self.idea2joke = dspy.ChainOfThought(IdeaToJoke)
        self.idea2joke.set_lm(dspy.LM('gemini/gemini-2.5-flash-lite', temperature=0.7))
        self.n_reflections = n_reflections
    
    async def acall(self, joke_idea: JokeIdea):
        joke = None
        for _ in range(self.n_reflections):
            joke = self.idea2joke(joke_idea=joke_idea, joke_draft=joke)
            print(joke)
        return joke.joke if joke is not None else ''

In [25]:
dspy.configure(lm=dspy.LM('gemini/gemini-2.5-flash-lite'), temperature=1)
dspy.configure_cache(enable_disk_cache=False, enable_memory_cache=False)

idea_gen = IdeaGenerator(n_samples=3)
joke_gen = JokeGenerator(n_reflections=2)

idea = await idea_gen.acall(query='OpenAI')
joke = await joke_gen.acall(joke_idea=idea)

Generated Joke Ideas: 
[Prediction(
    trajectory={'thought_0': 'I need to come up with a joke about OpenAI. The user\'s query is "OpenAI". I should fetch some recent news about OpenAI to get some ideas for a joke. The `fetch_recent_news` tool seems appropriate for this. I\'ll use "OpenAI" as the query.', 'tool_name_0': 'fetch_recent_news', 'tool_args_0': {'query': 'OpenAI'}, 'observation_0': ['# Musk lawsuit over OpenAI for-profit conversion can head to trial, US judge says. * Musk claims OpenAI violated its founding mission in restructuring to a for-profit entity. * OpenAI denies claims, calls Musk\'s lawsuit baseless. * Musk was a co-founder of OpenAI, now runs a rival AI company. WASHINGTON, Jan 7 (Reuters) - Billionaire entrepreneur Elon Musk persuaded a judge on Wednesday to allow a jury trial on his allegations that ChatGPT maker OpenAI violated its founding mission in its high-profile restructuring to a for-profit entity. Musk was a cofounder of OpenAI in 2015 but left in 2018

In [26]:
print(joke)

Alright, so you know OpenAI, right? These guys started with this super noble, heartwarming mission statement: "Let's make sure artificial general intelligence benefits *all* of humanity." Like, they were gonna be the AI fairy godmothers, sprinkling smartness on everyone. And get this – it was supposed to be *non-profit*! All about the greater good, sharing the AI wealth, you know?

Then, plot twist! Who decides to sue them? Elon Musk! Yeah, one of the original architects of OpenAI. He's basically saying, "Hey! You guys have gone totally corporate! Where's the open source? Where's the humanity? All I see is dollar signs!"

And you know what's hilarious? It turns out OpenAI's biggest, most earth-shattering innovation wasn't some mind-blowing AI... it was figuring out how to perfectly transform the word "Open" into "Own More Money."
